# Cryptographic Utilities Exercise || I.BA_MOVK_MM, Semesterweek 04
*Please write down to solution of the exercises in a consise but comprehensible way. Numerical results
should be accurate to 4 digits.*

## 1 Exercise 1: MD5, SHA1, SHA256, SHA512

Generate the previsously mentioned hash values of the string $Hello World!$. Do You get the same  
result as Your neighbour? Check the size of each hash value! What are the corresponding block sizes?  
Make a table! How many hash values would You have to generate in order to find a collision with a  
probability of at least 50% (90%)?  

---

We will use some good ol' Python for that

In [30]:
import hashlib

plaintext = b'HelloWorld!'

md5hash = hashlib.md5(plaintext)
print("md5: ",md5hash.hexdigest())
print("md5-size: ",md5hash.digest_size)
print("md5-block-size: ",md5hash.block_size)
print()

sha1hash = hashlib.sha1(plaintext)
print("sha1: ",sha1hash.hexdigest())
print("sha1-size: ",sha1hash.digest_size)
print("sha1-block-size: ",sha1hash.block_size)
print()


sha256hash = hashlib.sha256(plaintext)
print("sha256: ",sha256hash.hexdigest())
print("sha256-size: ",sha256hash.digest_size)
print("sha256-block-size: ",sha256hash.block_size)
print()


sha512hash = hashlib.sha512(plaintext)
print("sha512: ",sha512hash.hexdigest())
print("sha512-size: ",sha512hash.digest_size)
print("sha512-block-size: ",sha512hash.block_size)


md5:  06e0e6637d27b2622ab52022db713ce2
md5-size:  16
md5-block-size:  64

sha1:  d735871a64133ee062400659cf91b8234d1c1930
sha1-size:  20
sha1-block-size:  64

sha256:  729e344a01e52c822bdfdec61e28d6eda02658d2e7d2b80a9b9029f41e212dde
sha256-size:  32
sha256-block-size:  64

sha512:  c5179301e5619c392fb9b8872b90156ea45e452ea7336d9ee727d5d4a2a95df6a973ddfdeb92bbc14e01aa923591d44835ac32ccd1a71f4681f6a731fae5c238
sha512-size:  64
sha512-block-size:  128


The values are obviously the same as my neighbours, because it is a hash and not an encryption with different keys!    

The Probability of a Collission is besth shown here:  

<img src="img/md5col.png" style="height:400px">






## Exercise 2: True random number generator  

Check if Your computer can generate true random numbers, possibly through some special device? Find  
out the underlying physical principle (thermal noise, etc.). How can You use the random number generator  
in the power shell (Windows) or some shell (bash, sh, etc.) on Linux or MAC OS X?  
How are true random numbers generated on smart cards? Do some research on the internet!  

---
 
**Windows:** https://en.wikipedia.org/wiki/CryptGenRandom  
**True-Randomness**: Pseudorandom 

---

**Linux**: /dev/random  
**True-Randomness**: Verwendet den "Noise" von Systemtreibern (CPU clock, GPU clock, Voltage Level...)  
**Example**: cat /dev/urandom > ~/urandom_test2   

--- 

**Smartcard:** Almost all - if not all - smart cards have a true random number generator (TRNG) on it. Most often the TRNG depends on thermal noise or clock drift, but other entropy sources are possible. A more technical evaluation of this can be found e.g. here   
[https://csrc.nist.gov/csrc/media/events/random-bit-generation-workshop-2016/documents/presentations/sessionvi-2-viktor-fischer-presentation.pdf])  
**True Randomness**: Allgemein werden "Hardware Random Number Generators" (hrng) verwendet bei smartcards
https://en.wikipedia.org/wiki/Hardware_random_number_generator.  
Diese messen "Noise" wie Thermisches Rauschen oder den Photoelektrischen Effekt.


## Exercise 3: Linear congruential random number generator

Compute the period of a linear congruential number generator if $m$ = 11, $b$ = 5, $a$ = 3 and $x_{0}$ = 7 is  
used. Write down the sequence of random number! What is the maximal size of the period? Adapt the paramters, such that the period is maximal!  

---

Formel gemäss Folie Nr 37 :

$x_{0} = s_{0} = 7$

$s_{i} = (a \cdot s_{i-1} + b) \mod m$  

**Example**

$s_{1} = (3 \cdot s_{0} + 5) \mod 11 = (3 \cdot 7 + 5) \mod 11 = 4$   

Das Maximum können wir mittels Python berechnen:

In [42]:
def lcrn(a, sx, b, m):
    res = ((a * sx + b) % m)
    print(res)
    return res
    

sx = lcrn(3,7,5,11)
x = 1
while x<22:
    sx = lcrn(3,sx,5,11)
    x=x+1

4
6
1
8
7
4
6
1
8
7
4
6
1
8
7
4
6
1
8
7
4
6


Die momentane Period ist bei **5** (nach 5 wiederholt sich das Muster.

---

Um die Period maximal zu machen muss folgendes gelten:  
- gcd(b,m) = 1 (erfüllt)
- jeder primfaktor von m dividiert a-1 (nicht erfüllt)
- falls eine Zahl m teilt, teilt diese Zahl auch a-1 (nicht erfüllt)

Um diese Bedingungen zu erfüllen müssten wir also lediglich **a** anpassen, z.B. auf 12

In [44]:
sx = lcrn(12,7,5,11)
x = 1
while x<22:
    sx = lcrn(12,sx,5,11)
    x=x+1

1
6
0
5
10
4
9
3
8
2
7
1
6
0
5
10
4
9
3
8
2
7


Mit unserer angepassten Formel erhalten wir nun eine Maximale Periode von 11!

**Note von Dave:** Bin mir nicht sicher ob diese Lösung so stimmt, danke fürs kontrollieren

## Aufgabe 4: LFSR  

Construct an LFSR of length 5 which produces the sequence of bits (1, 0, 1, 1, 0, 0, 1, 0, 1, 0,...). What
is the period of this LFSR? What is the maximal period? Try to improve this LFSR such that it has maximum period!  

**Python Way**:

In [47]:
def lfsr5(seed, taps):
    sr, xor = seed, 0
    while 1:
        for t in taps:
            xor += int(sr[t-1])
        xor %= 2
        sr, xor = str(xor) + sr[:-1], 0
        yield sr
        if sr == seed:
            break
                
a = lfsr5('10110', (5,3,2,1)) #IV
[next(a) for j in range(31)]

['01011',
 '00101',
 '00010',
 '00001',
 '10000',
 '11000',
 '01100',
 '00110',
 '10011',
 '01001',
 '00100',
 '10010',
 '11001',
 '11100',
 '11110',
 '11111',
 '01111',
 '10111',
 '11011',
 '11101',
 '01110',
 '00111',
 '00011',
 '10001',
 '01000',
 '10100',
 '01010',
 '10101',
 '11010',
 '01101',
 '10110']

Period is 15 which can be improved to 31 by adding c2 = 1 as an example! To make computations easy,
use the following python code

**Papier Way:**


<img src="img/psol.jpg" style="height:400px">


## Exercise 4:Is random really random (optional)

Nope

## Exercise 5:Probability for a collision

There are 40 people in a room. You bet, that there are at least 2 people with the same birthday. What is the probability, that You win? Use the exact formula as well as the approximation



In [7]:
from sympy import *
i = symbols('i',integer=True)
p = 1 - product((365-i)/365,(i,1,39))
p.evalf()

0.891231809817949

**Formel:**


$ 1 - \frac{365!}{(365 - n)! \cdot 365^{n}} =  1 - \frac{365!}{(365 - 40)! \cdot 365^{40}} = 0.891$
  
**Mathematischer Hintergrund**: Im **Urnenmodell** entspricht diese Annahme einer Ziehung von $n$ Kugeln mit Zurücklegen aus einer Urne, die 365 Kugeln mit der Beschriftung „1. Januar“, „2. Januar“ usw. bis „31. Dezember“ enthält. 

## Exercise 6:Probability of a collision

Suppose You use a hash function of length 128 bits. How many hash values would You have to compute in order to find a collision with probability at least 90%?   Use the approximate forumula from Slide 14/58.  

$n = 2^{(m+1)/2} \cdot \sqrt{(\ln{(\frac{1}{1-p})}}$  

where m=128 and p=0.9.

In [13]:
import math
import numpy as np
m = 128
p = 0.9

n = 2**((m+1)/2) * ( math.sqrt(np.log(1/(1-p)) ) )
print (n)

3.958608607783506e+19


How much storage would You need, if You need 16 bytes for each hash?

In [17]:
byte = n * 16
kilobyte = byte/1024
megabyte = kilobyte/1024
gigabyte = megabyte/1024
terabyte = gigabyte/1024
petabyte = terabyte/1024
exabyte = exabyte/1024



print("byte: ",byte)
print("kilobyte: ",kilobyte)
print("megabyte: ",megabyte)
print("gigabyte: ",gigabyte)
print("terabyte: ",terabyte)
print("petabyte: ",petabyte)

byte:  6.33377377245361e+20
kilobyte:  6.185325949661728e+17
megabyte:  604035737271653.1
gigabyte:  589878649679.3488
terabyte:  576053368.827489
petabyte:  562552.1179955947
